In [46]:
import csv
import os
import datetime as datetime
from datetime import datetime, timezone
import pandas as pd

In [5]:
user="TylerTwohig"

user_input = """
Hello my name is Tyler Twohig. 
I am a master's student in Information Management & Systems 
at the University of California, Berkeley
"""

model = "DeepSeek-R1:latest"

In [7]:
def get_iso_time():
    now = datetime.now(timezone.utc)
    timestamp_iso = now.isoformat(timespec="milliseconds")
    timestamp_iso = timestamp_iso.replace("+00:00", "Z")
    timestamp_ms = int(now.timestamp()*1000)

    return timestamp_iso, timestamp_ms

In [222]:
#check if there is an existing row in the input history for user where the same input was already used on the same model
#returns row where user_input and model combination already exist

def check_existing_input(user_input, model, filename):
    if not os.path.isfile(filename):
        return None

    df = pd.read_csv(filename)

    matches = df.loc[(df['model'] == model) & (df['user_input'] == user_input)].copy()

    return matches if not matches.empty else None

In [305]:
def save_user_input(user_input, model, filename=user+"_input_history.csv"):
    
    if not os.path.isfile(filename):
        with open(filename, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow([
                'timestamp_iso',
                'timestamp_ms',
                'model',
                'user_input',
                'processed'
            ])

    existing_row = check_existing_input(user_input, model, filename)
    
    if existing_row is not None:
        dt = existing_row['timestamp_ms'].iloc[0]
        dt = datetime.fromtimestamp(dt / 1000)
        print(f"Input already used at: {dt}")
        return False


    timestamp_iso, timestamp_ms = get_iso_time()

    with open(filename, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow([
            timestamp_iso,
            timestamp_ms,
            model,
            user_input,
            0 #default=0, new inputs have not been processed yet
        ])

    return True

In [309]:
save_user_input("s", model)

Input already used at: 2025-08-30 17:17:54.876000


False